## Omics Analysis with Apache Spark
### DANN編

今回はDANNのデータをApache Sparkのデータフレームへ取り込んでいきます。   
DANNは遺伝子変異の病原性をスコアリングしたもので、深層学習が使われています。


### データの下準備
予め以下のデータをダウンロードしておいてあります。 
103GBあります。
```
$ ls -sh DANN_whole_genome_SNVs.tsv.bgz 
103G DANN_whole_genome_SNVs.tsv.bgz
```
最初の10行を見てみます。
```
$ zcat DANN_whole_genome_SNVs.tsv.bgz|head
1	10001	T	A	0.16461391399220135
1	10001	T	C	0.4396994049749739
1	10001	T	G	0.38108629377072734
1	10002	A	C	0.36182020272810128
1	10002	A	G	0.44413258111779291
1	10002	A	T	0.16812846819989813
1	10003	A	C	0.36516159615040267
1	10003	A	G	0.4480978029675266
1	10003	A	T	0.17573150952125288
1	10004	C	A	0.16559066203780862
```

とてもシンプルなデータですね。そしてheaderはありません。  
chromosome, position, ref, alt, DANN_score  
という名前でheaderを作ることにします。

データはhdfsに置きました。

### データの読み込み
inferSchema=Falseで読み込みます。  
headerもないので、header=Falseです。

In [ ]:
delta_output_path = 'hdfs://hostname:port/path-to-data/DANN-delta'

In [ ]:
df = spark.read.option("delimiter","\t")\
    .csv("hdfs://hostname:port/path-to-data/DANN/DANN_whole_genome_SNVs.tsv.bgz",inferSchema=False,nullValue="NA",header=False)

表示させてみます

In [ ]:
df.show()

schemaをつくり、適用させて読み込んでみます。

In [ ]:
from pyspark.sql.types import *

schema = StructType([
  StructField("chromosome", StringType(), False),
  StructField("position", IntegerType(), False),
  StructField("ref", StringType(), False),
  StructField("alt", StringType(), False),
  StructField("DANN_score", DoubleType(), False)
])

In [ ]:
%%time
df2 = spark.read.option("delimiter","\t")\
    .csv("hdfs://hostname:port/path-to-data/DANN/DANN_whole_genome_SNVs.tsv.bgz",inferSchema=False,nullValue="NA",header=False,schema=schema).cache()
df2.count()

In [ ]:
df2.show()

In [ ]:
df2.printSchema()

良さそうです。  

### DataFrame操作
ここではRawScoreでsortしています。

In [ ]:
%%time
display(df2.sort("DANN_score").select("chromosome","position","ref","alt","DANN_score").limit(20).toPandas())

スコアの高い順番にsortする場合は次のようにdesc()で項目名を囲みます。

In [ ]:
%%time
from pyspark.sql.functions import desc
display(df2.sort(desc("DANN_score")).select("chromosome","position","ref","alt","DANN_score").limit(20).toPandas())